In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

In [3]:
#import the dataset and gain some info on it 
data = pd.read_csv('F:/Data science/projects/data_analysis/titanic_project/data/Titanic-Dataset.csv')

#check the shape and info of the data 
data.info()

#print the shape 
print(data.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
(891, 12)


In [4]:
#check if there are any null values 
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [12]:
#drop any missing values 
#we are going to perform data cleaning 

def data_clean(data):
    df = data.copy()
    #remove the column with many missing features 
    df = df.drop(columns = ['PassengerId', 'Cabin', 'Ticket', 'Name'])
    #df fill missing values 
    df['Embarked'].fillna('S', inplace=True)

    #df map the sex with 1 and zero 
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

    #encode the embarked 
    df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)
    #fill the age of missing values with middle age 
    df['Age'].fillna(df['Age'].median(), inplace=True)

    #calculate the family sise 
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    
    return df






In [13]:
#generate the new dataframe 
df = data_clean(data)
df.head()

C:\Users\Chris Victor\AppData\Local\Temp\ipykernel_12424\1446167015.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Embarked'].fillna('S', inplace=True)
C:\Users\Chris Victor\AppData\Local\Temp\ipykernel_12424\1446167015.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,FamilySize
0,0,3,0,22.0,1,0,7.2500,False,True,2
1,1,1,1,38.0,1,0,71.2833,False,False,2
2,1,3,1,26.0,0,0,7.9250,False,True,1
3,1,1,1,35.0,1,0,53.1000,False,True,2
4,0,3,0,35.0,0,0,8.0500,False,True,1
